In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 18:31:20.687325 140649269012288 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 18:31:20.695386 140649269012288 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 18:31:20.700241 140649269012288 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [10]:
def train(model, epochs=200, patience=10):
    """
    """
    wait = 0
    preds = None
    best_val_loss = 99999
    # Fit
    for epoch in range(1, epochs + 1):

        # Log wall-clock time
        t = time.time()

        # Single training iteration (we mask nodes without labels for loss calculation)
        model.fit(graph, y_train, sample_weight=train_mask,
                  batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

        # Predict on full dataset
        preds = model.predict(graph, batch_size=A.shape[0])

        # Train / validation scores
        train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                       [idx_train, idx_val])
        print("Epoch: {:04d}".format(epoch),
              "train_loss= {:.4f}".format(train_val_loss[0]),
              "train_acc= {:.4f}".format(train_val_acc[0]),
              "val_loss= {:.4f}".format(train_val_loss[1]),
              "val_acc= {:.4f}".format(train_val_acc[1]),
              "time= {:.4f}".format(time.time() - t))

        # Early stopping
        if train_val_loss[1] < best_val_loss:
            best_val_loss = train_val_loss[1]
            wait = 0
        else:
            if wait >= patience:
                print('Epoch {}: early stopping'.format(epoch))
                break
            wait += 1
    return preds

# GCN (baseline)

In [11]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 18:31:20.715382 140649269012288 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 18:31:20.719588 140649269012288 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 18:31:20.727949 140649269012288 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [12]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 18:31:20.769330 140649269012288 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [14]:
preds = train(model)

W1110 18:31:20.846734 140649269012288 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9315 train_acc= 0.3000 val_loss= 1.9319 val_acc= 0.3500 time= 1.1493
Epoch: 0002 train_loss= 1.9150 train_acc= 0.3000 val_loss= 1.9176 val_acc= 0.3500 time= 0.0173
Epoch: 0003 train_loss= 1.8965 train_acc= 0.3429 val_loss= 1.9023 val_acc= 0.3633 time= 0.0168
Epoch: 0004 train_loss= 1.8769 train_acc= 0.3786 val_loss= 1.8860 val_acc= 0.3633 time= 0.0164
Epoch: 0005 train_loss= 1.8566 train_acc= 0.3857 val_loss= 1.8691 val_acc= 0.3633 time= 0.0165
Epoch: 0006 train_loss= 1.8355 train_acc= 0.3857 val_loss= 1.8518 val_acc= 0.3667 time= 0.0164
Epoch: 0007 train_loss= 1.8147 train_acc= 0.3929 val_loss= 1.8351 val_acc= 0.3667 time= 0.0175
Epoch: 0008 train_loss= 1.7948 train_acc= 0.4071 val_loss= 1.8193 val_acc= 0.3667 time= 0.0177
Epoch: 0009 train_loss= 1.7758 train_acc= 0.4071 val_loss= 1.8046 val_acc= 0.3667 time= 0.0170
Epoch: 0010 train_loss= 1.7575 train_acc= 0.4000 val_loss= 1.7905 val_acc= 0.3667 time= 0.0172
Epoch: 0011 train_loss= 1.7405 train_acc= 0.4071 v

Epoch: 0097 train_loss= 0.6137 train_acc= 0.9357 val_loss= 0.9365 val_acc= 0.7900 time= 0.0165
Epoch: 0098 train_loss= 0.6074 train_acc= 0.9500 val_loss= 0.9322 val_acc= 0.7900 time= 0.0167
Epoch: 0099 train_loss= 0.6016 train_acc= 0.9500 val_loss= 0.9278 val_acc= 0.7900 time= 0.0166
Epoch: 0100 train_loss= 0.5960 train_acc= 0.9500 val_loss= 0.9233 val_acc= 0.7900 time= 0.0163
Epoch: 0101 train_loss= 0.5901 train_acc= 0.9500 val_loss= 0.9184 val_acc= 0.7967 time= 0.0166
Epoch: 0102 train_loss= 0.5845 train_acc= 0.9500 val_loss= 0.9132 val_acc= 0.8033 time= 0.0162
Epoch: 0103 train_loss= 0.5796 train_acc= 0.9500 val_loss= 0.9084 val_acc= 0.8000 time= 0.0168
Epoch: 0104 train_loss= 0.5753 train_acc= 0.9500 val_loss= 0.9040 val_acc= 0.7933 time= 0.0167
Epoch: 0105 train_loss= 0.5706 train_acc= 0.9500 val_loss= 0.9005 val_acc= 0.7933 time= 0.0163
Epoch: 0106 train_loss= 0.5659 train_acc= 0.9571 val_loss= 0.8973 val_acc= 0.7933 time= 0.0164
Epoch: 0107 train_loss= 0.5608 train_acc= 0.9571 v

Epoch: 0194 train_loss= 0.3355 train_acc= 0.9857 val_loss= 0.7340 val_acc= 0.8267 time= 0.0179
Epoch: 0195 train_loss= 0.3342 train_acc= 0.9714 val_loss= 0.7309 val_acc= 0.8333 time= 0.0169
Epoch: 0196 train_loss= 0.3342 train_acc= 0.9786 val_loss= 0.7295 val_acc= 0.8233 time= 0.0175
Epoch: 0197 train_loss= 0.3339 train_acc= 0.9786 val_loss= 0.7301 val_acc= 0.8233 time= 0.0171
Epoch: 0198 train_loss= 0.3312 train_acc= 0.9786 val_loss= 0.7306 val_acc= 0.8233 time= 0.0176
Epoch: 0199 train_loss= 0.3285 train_acc= 0.9786 val_loss= 0.7326 val_acc= 0.8267 time= 0.0167
Epoch: 0200 train_loss= 0.3292 train_acc= 0.9857 val_loss= 0.7403 val_acc= 0.8200 time= 0.0166


In [16]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7685
accuracy = 0.8220


# Gaussian Graph Convolution

In [17]:
B1 = 5e-4
B2 = 5e-4

In [18]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
#     dropout=0.5
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 18:31:25.422359 140649269012288 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:174: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 18:31:25.423832 140649269012288 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should updat

In [19]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)

W1110 18:31:25.458773 140649269012288 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:28: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [21]:
preds = train(model)

Epoch: 0001 train_loss= 1.9409 train_acc= 0.4357 val_loss= 1.9434 val_acc= 0.3167 time= 0.4739
Epoch: 0002 train_loss= 1.9366 train_acc= 0.4929 val_loss= 1.9409 val_acc= 0.4033 time= 0.0170
Epoch: 0003 train_loss= 1.9303 train_acc= 0.6071 val_loss= 1.9358 val_acc= 0.4900 time= 0.0163
Epoch: 0004 train_loss= 1.9223 train_acc= 0.6571 val_loss= 1.9319 val_acc= 0.4967 time= 0.0163
Epoch: 0005 train_loss= 1.9165 train_acc= 0.6143 val_loss= 1.9257 val_acc= 0.5167 time= 0.0163
Epoch: 0006 train_loss= 1.9082 train_acc= 0.5714 val_loss= 1.9200 val_acc= 0.5200 time= 0.0164
Epoch: 0007 train_loss= 1.8970 train_acc= 0.6357 val_loss= 1.9139 val_acc= 0.5233 time= 0.0165
Epoch: 0008 train_loss= 1.8870 train_acc= 0.6357 val_loss= 1.9059 val_acc= 0.5333 time= 0.0165
Epoch: 0009 train_loss= 1.8774 train_acc= 0.6143 val_loss= 1.9018 val_acc= 0.5100 time= 0.0162
Epoch: 0010 train_loss= 1.8666 train_acc= 0.6429 val_loss= 1.8931 val_acc= 0.5500 time= 0.0163
Epoch: 0011 train_loss= 1.8556 train_acc= 0.6500 v

Epoch: 0098 train_loss= 0.7427 train_acc= 0.9071 val_loss= 1.0282 val_acc= 0.8000 time= 0.0187
Epoch: 0099 train_loss= 0.7367 train_acc= 0.9071 val_loss= 1.0224 val_acc= 0.8000 time= 0.0180
Epoch: 0100 train_loss= 0.7299 train_acc= 0.9071 val_loss= 1.0172 val_acc= 0.8000 time= 0.0164
Epoch: 0101 train_loss= 0.7236 train_acc= 0.9071 val_loss= 1.0116 val_acc= 0.8033 time= 0.0166
Epoch: 0102 train_loss= 0.7178 train_acc= 0.9071 val_loss= 1.0073 val_acc= 0.8000 time= 0.0163
Epoch: 0103 train_loss= 0.7119 train_acc= 0.9071 val_loss= 1.0028 val_acc= 0.8033 time= 0.0164
Epoch: 0104 train_loss= 0.7063 train_acc= 0.9071 val_loss= 0.9980 val_acc= 0.8000 time= 0.0164
Epoch: 0105 train_loss= 0.6999 train_acc= 0.9071 val_loss= 0.9936 val_acc= 0.8033 time= 0.0166
Epoch: 0106 train_loss= 0.6954 train_acc= 0.9143 val_loss= 0.9893 val_acc= 0.8033 time= 0.0165
Epoch: 0107 train_loss= 0.6882 train_acc= 0.9214 val_loss= 0.9854 val_acc= 0.8033 time= 0.0172
Epoch: 0108 train_loss= 0.6829 train_acc= 0.9143 v

Epoch: 0185 train_loss= 0.4315 train_acc= 0.9643 val_loss= 0.7907 val_acc= 0.8100 time= 0.0167
Epoch: 0186 train_loss= 0.4298 train_acc= 0.9643 val_loss= 0.7891 val_acc= 0.8067 time= 0.0167
Epoch: 0187 train_loss= 0.4270 train_acc= 0.9714 val_loss= 0.7870 val_acc= 0.8100 time= 0.0166
Epoch: 0188 train_loss= 0.4241 train_acc= 0.9714 val_loss= 0.7850 val_acc= 0.8100 time= 0.0167
Epoch: 0189 train_loss= 0.4230 train_acc= 0.9714 val_loss= 0.7837 val_acc= 0.8167 time= 0.0163
Epoch: 0190 train_loss= 0.4204 train_acc= 0.9714 val_loss= 0.7830 val_acc= 0.8133 time= 0.0164
Epoch: 0191 train_loss= 0.4186 train_acc= 0.9714 val_loss= 0.7817 val_acc= 0.8167 time= 0.0164
Epoch: 0192 train_loss= 0.4162 train_acc= 0.9714 val_loss= 0.7822 val_acc= 0.8200 time= 0.0163
Epoch: 0193 train_loss= 0.4147 train_acc= 0.9714 val_loss= 0.7811 val_acc= 0.8200 time= 0.0166
Epoch: 0194 train_loss= 0.4129 train_acc= 0.9714 val_loss= 0.7793 val_acc= 0.8200 time= 0.0163
Epoch: 0195 train_loss= 0.4112 train_acc= 0.9714 v

In [23]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8267
accuracy = 0.8100


# Attacks